In [2]:
from google.colab import drive
drive.mount("/content/gdive")

Mounted at /content/gdive


In [3]:
%cd /content/gdive/MyDrive/Colab Notebooks/stdcode2024/codeset

/content/gdive/MyDrive/Colab Notebooks/stdcode2024/codeset


In [47]:
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
from sklearn import tree
from sklearn import ensemble
from sklearn.tree import plot_tree

# 모델 정확도 라이브러리 선언
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

# CSV 파일을 읽어 DataFrame 변수에 저장하기
fdata = pd.read_csv("../dataset_out/Property_df.csv")
fdata["year"] = fdata['Year/Month'].str[:4]
fdata['year']=fdata['year'].astype("int")
fdata.dtypes

Year/Month     object
전국            float64
수도권           float64
지방            float64
서울            float64
(서울)도심권       float64
(서울)동북권       float64
(서울)서북권       float64
(서울)서남권       float64
(서울)동남권       float64
부산            float64
금리            float64
year            int64
dtype: object

In [48]:
corrDf = fdata.corr(numeric_only=True)
corrDf

,전국,수도권,지방,서울,(서울)도심권,(서울)동북권,(서울)서북권,(서울)서남권,(서울)동남권,부산,금리,year
전국,1.000000,0.934795,0.927029,0.862261,0.868067,0.842118,0.882909,0.871464,0.857886,0.923361,-0.547933,0.899481
수도권,0.934795,1.000000,0.878951,0.971384,0.959179,0.955801,0.970733,0.967541,0.958411,0.900913,-0.480246,0.889603
지방,0.927029,0.878951,1.000000,0.845113,0.849070,0.840668,0.871760,0.851345,0.841809,0.987750,-0.612492,0.974000
서울,0.862261,0.971384,0.845113,1.000000,0.986427,0.990967,0.992583,0.995968,0.992771,0.879296,-0.430953,0.879565
(서울)도심권,0.868067,0.959179,0.849070,0.986427,1.000000,0.981763,0.986042,0.987237,0.984388,0.882979,-0.424227,0.880249
(서울)동북권,0.842118,0.955801,0.840668,0.990967,0.981763,1.000000,0.992532,0.991545,0.985060,0.870724,-0.447831,0.876465
(서울)서북권,0.882909,0.970733,0.871760,0.992583,0.986042,0.992532,1.000000,0.993772,0.989363,0.899338,-0.453231,0.899045
(서울)서남권,0.871464,0.967541,0.851345,0.995968,0.987237,0.991545,0.993772,1.000000,0.990996,0.883528,-0.439844,0.883498
(서울)동남권,0.857886,0.958411,0.841809,0.992771,0.984388,0.985060,0.989363,0.990996,1.000000,0.877794,-0.417796,0.881520
부산,0.923361,0.900913,0.987750,0.879296,0.882979,0.870724,0.899338,0.883528,0.877794,1.000000,-0.553565,0.978116


In [49]:
featuresStd = 0.5
label = '금리'
features = list(corrDf [ (abs(corrDf[label]) > featuresStd) &
                              (abs(corrDf[label]) != 1)].index)
features

['전국', '지방', '부산', 'year']

In [50]:
splitRatio = 0.8
stdYear = int(fdata["year"].min() + (fdata["year"].max()-fdata["year"].min())*splitRatio)
stdYear

2020

In [80]:
trainingDataFeatures = fdata.loc[ fdata.year <= stdYear, features ]
trainingDataLabel = fdata.loc[ fdata.year <= stdYear, label ]
testDataFeatures = fdata.loc[ fdata.year > stdYear, features ]
testDataLabel = fdata.loc[ fdata.year > stdYear, label ]

In [81]:
# model : DecisionTree
modelDt = tree.DecisionTreeRegressor(random_state=10)
fittedDtModel = modelDt.fit(trainingDataFeatures,
                            trainingDataLabel )
# model : RandomForest
modelRf = ensemble.RandomForestRegressor(random_state=10)
fittedRfModel = modelRf.fit(trainingDataFeatures,
                            trainingDataLabel )

In [82]:
print(trainingDataFeatures.shape)
print(trainingDataLabel.shape)
print(testDataFeatures.shape)
print(testDataLabel.shape)

(180, 4)
(180,)
(38, 4)
(38,)


In [83]:
# dt모델 예측 정확도
mae_dt = mean_absolute_error(testDataLabel, predictValueDt)
mse_dt = mean_squared_error(testDataLabel, predictValueDt)
rmse_dt = np.sqrt(mse_dt)
accuracy_dt = fittedDtModel.score(testDataFeatures, testDataLabel)
accuracy_dt_per = (accuracy_dt*100) - (accuracy_dt*100)%0.01

print(f"MSE DT: {mae_dt}")
print(f"MSE DT: {mse_dt}")
print(f"RMSE DT: {rmse_dt}")
print(f"Accuracy DT: {accuracy_dt_per}%")

MSE DT: 1.5986842105263157
MSE DT: 4.192434210526316
RMSE DT: 2.047543457542798
Accuracy DT: -167.04%


In [85]:
# rf모델 예측 정확도
mae_rf = mean_absolute_error(testDataLabel, predictValueRf)
mse_rf = mean_squared_error(testDataLabel, predictValueRf)
rmse_rf = np.sqrt(mse_rf)
accuracy_rf = fittedRfModel.score(testDataFeatures, testDataLabel)
accuracy_rt_per = (accuracy_rf*100) - (accuracy_rf*100)%0.01

print(f"MSE RF: {mae_rf}")
print(f"MSE RF: {mse_rf}")
print(f"RMSE RF: {rmse_rf}")
print(f"Accuracy RF: {accuracy_rt_per}%")

MSE RF: 1.5607894736842105
MSE RF: 4.012694407894737
RMSE RF: 2.0031710880238704
Accuracy RF: -155.59%
